## OpenAI 

For this notebook we will use the OpenAI ChatGPT models. We import the OpenAI agent and set the api_key, you will have to provide your own API key. 

In [ ]:
# Setup OpenAI Agent
import openai

openai.api_key = "sk-your-key"
from llama_index.agent import OpenAIAgent

## Database tool

This tool connects to a database (using SQLAlchemy under the hood) and allows an Agent to query the database and get information about the tables.

We import the ToolSpec and initialize it so that it can connect to our database

In [ ]:
# Import and initialize our tool spec
from llama_index.tools.database.base import DatabaseToolSpec

db_spec = DatabaseToolSpec(
    scheme="postgresql",  # Database Scheme
    host="localhost",  # Database Host
    port="5432",  # Database Port
    user="postgres",  # Database User
    password="x",  # Database Password
    dbname="your_db",  # Database Name
)

After initializing the Tool Spec we have an instance of the ToolSpec. A ToolSpec can have many tools that it implements and makes available to agents. We can see the Tools by converting to the spec to a list of FunctionTools, using `to_tool_list`

In [ ]:
tools = db_spec.to_tool_list()
for tool in tools:
    print(tool.metadata.name)
    print(tool.metadata.description)
    print(tool.metadata.fn_schema)

load_data
load_data(query: str) -> List[llama_index.schema.Document]
Query and load data from the Database, returning a list of Documents.

        Args:
            query (str): an SQL query to filter tables and rows.

        Returns:
            List[Document]: A list of Document objects.
        
<class 'pydantic.main.load_data'>
describe_tables
describe_tables(tables: Optional[List[str]] = None) -> str

            Describes the specifed tables in the database

            Args:
                tables (List[str]): A list of table names to retrieve details about
        
<class 'pydantic.main.describe_tables'>
list_tables
list_tables() -> List[str]

            Returns a list of available tables in the database.
            To retrieve details about the columns of specfic tables, use
            the describe_tables endpoint
        
<class 'pydantic.main.list_tables'>


We can see that the database tool spec provides 3 functions for the OpenAI agent. One to execute a SQL query, one to describe a list of tables in the database, and one to list all of the tables available in the database. 

We can pass the tool list to our OpenAI agent and test it out:

In [ ]:
# Create the Agent with our tools
agent = OpenAIAgent.from_tools(tools, verbose=True)

At this point our Agent is fully ready to start making queries to our database:

In [ ]:
agent.chat("What tables does this database contain")

=== Calling Function ===
Calling function: list_tables with args: {}
Got output: ['active_storage_blobs', 'addresses', 'amenities', 'amenities_dwellings', 'ar_internal_metadata', 'dwellings', 'listings', 'rooms', 'schema_migrations', 'users', 'waitlists', 'active_storage_attachments', 'active_storage_variant_records', 'messages', 'participants']


Response(response='This database contains the following tables:\n1. active_storage_blobs\n2. addresses\n3. amenities\n4. amenities_dwellings\n5. ar_internal_metadata\n6. dwellings\n7. listings\n8. rooms\n9. schema_migrations\n10. users\n11. waitlists\n12. active_storage_attachments\n13. active_storage_variant_records\n14. messages\n15. participants', source_nodes=[], metadata=None)

In [ ]:
agent.chat("Can you describe the messages table")

=== Calling Function ===
Calling function: describe_tables with args: {
  "tables": ["messages"]
}
Got output: 
CREATE TABLE messages (
	id BIGSERIAL NOT NULL, 
	user_id INTEGER NOT NULL, 
	room_id INTEGER NOT NULL, 
	body TEXT, 
	created_at TIMESTAMP(6) WITHOUT TIME ZONE NOT NULL, 
	updated_at TIMESTAMP(6) WITHOUT TIME ZONE NOT NULL, 
	CONSTRAINT messages_pkey PRIMARY KEY (id), 
	CONSTRAINT fk_rails_273a25a7a6 FOREIGN KEY(user_id) REFERENCES users (id), 
	CONSTRAINT fk_rails_a8db0fb63a FOREIGN KEY(room_id) REFERENCES rooms (id)
)





Response(response='The `messages` table has the following columns:\n\n1. `id`: A unique identifier for each message (type: BIGSERIAL)\n2. `user_id`: The ID of the user who sent the message (type: INTEGER)\n3. `room_id`: The ID of the room where the message was sent (type: INTEGER)\n4. `body`: The content of the message (type: TEXT)\n5. `created_at`: The timestamp when the message was created (type: TIMESTAMP(6) WITHOUT TIME ZONE)\n6. `updated_at`: The timestamp when the message was last updated (type: TIMESTAMP(6) WITHOUT TIME ZONE)\n\nThe table also has the following constraints:\n- `messages_pkey`: Primary key constraint on the `id` column\n- `fk_rails_273a25a7a6`: Foreign key constraint on the `user_id` column, referencing the `id` column of the `users` table\n- `fk_rails_a8db0fb63a`: Foreign key constraint on the `room_id` column, referencing the `id` column of the `rooms` table', source_nodes=[], metadata=None)

In [ ]:
agent.chat("Fetch the most recent message and display the body")

=== Calling Function ===
Calling function: load_data with args: {
  "query": "SELECT body FROM messages ORDER BY created_at DESC LIMIT 1"
}
Got output: [Document(id_='10c3ba99-d12e-4557-805d-c7e95a9b1ee7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='80a5d3ba4f8f87aaac2096195bbd1d81f9ac58c75d85413da2d9fa533335789e', text='cool', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')]


Response(response='The most recent message has the following body: "cool".', source_nodes=[], metadata=None)